#### 붓꽃 품종 분류
- 목표 : 붓꽃의 3개 품종을 분류하기
- 데이터셋 : 내장 데이터셋
- 피쳐 : 4개
- 타겟 : 품종 1개
- 학습 : 지도학습 > 분류

[1] 데이터 준비

In [138]:
# 모듈 로딩
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt

In [139]:
# 내장 데이터셋 로딩
data = load_iris(as_frame=True)

In [140]:
# Bunch 인스턴스 = >dict와 유사한 형태
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [141]:
featureDF=data['data']
targetSR=data['target']

In [142]:
featureDF.shape, targetSR.shape

((150, 4), (150,))

In [143]:
featureDF.head(1), targetSR.head(1)

(   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
 0                5.1               3.5                1.4               0.2,
 0    0
 Name: target, dtype: int32)

[2] 학습을 위한 데이터 셋 준비 => 학습용, 검증용, 테스트용 

In [144]:
#학습용 & 테스트용 분리
x_train,x_test,y_train,y_test=train_test_split(featureDF,
                                               targetSR,
                                               stratify=targetSR)

In [145]:
#학습용 & 검증용 분리
x_train,x_val,y_train,y_val=train_test_split(x_train,
                                               y_train,
                                               stratify=y_train)

In [146]:
print(f'Train DS: {x_train.shape} {x_train.shape[0]/featureDF.shape[0]:.2f}%')
print(f'val DS: {x_val.shape} {x_val.shape[0]/featureDF.shape[0]:.2f}%')
print(f'Test DS: {x_test.shape} {x_test.shape[0]/featureDF.shape[0]:.2f}%')

Train DS: (84, 4) 0.56%
val DS: (28, 4) 0.19%
Test DS: (38, 4) 0.25%


[3]교차검증 방식

In [147]:
# 모듈로드
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

In [148]:
# 모델 인스턴스 생성
dtc_model=DecisionTreeClassifier()

# [3-1] KFold 기반---------------------------------
# 정확도 저장 리스트
accuracys=[]

# KFold 인스턴스 생성 [기본 k=5]
kfold=KFold()


In [149]:
# k번 만큼 k개 데이터셋으로 학습 진행
# -> k등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스
for idx, (train_index, val_index) in enumerate(kfold.split(featureDF),1):
    
	#print(f'train_index: {train_index.tolist()}')

	# x_train, x_val 데이터셋 설정
	x_train, y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
	x_val,y_val = featureDF.iloc[val_index.tolist()],targetSR[val_index.tolist()]

	#학습진행
	dtc_model.fit(x_train, y_train)

	# 평가 => 분류의 경우 score()메서드 => 정확도 반환
	train_acc=dtc_model.score(x_train,y_train)
	val_acc=dtc_model.score(x_val,y_val)

	accuracys.append([train_acc,val_acc])
	print(f'[{idx}번째] train 정확도 : {train_acc} val 정확도 : {val_acc}')



[1번째] train 정확도 : 1.0 val 정확도 : 1.0
[2번째] train 정확도 : 1.0 val 정확도 : 0.9666666666666667
[3번째] train 정확도 : 1.0 val 정확도 : 0.8333333333333334
[4번째] train 정확도 : 1.0 val 정확도 : 0.9333333333333333
[5번째] train 정확도 : 1.0 val 정확도 : 0.8


In [150]:
# 평균 계산
train_mean = sum([value[0] for value in accuracys])/kfold.n_splits
test_mean = sum([value[1] for value in accuracys])/kfold.n_splits

print(f'Train 정확도 : {train_mean}   val정확도: {test_mean:.2f}')

Train 정확도 : 1.0   val정확도: 0.91


In [151]:
#### ===> [3-2] StraitfiedKFold
accuracys=[]


skFold=StratifiedKFold()

# k번 만큼 k개 데이터셋으로 학습 진행
# -> k등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스
for idx, (train_index, val_index) in enumerate(skFold.split(featureDF,targetSR),1):
    
	#print(f'train_index: {train_index.tolist()}')

	# x_train, x_val 데이터셋 설정
	x_train, y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
	x_val,y_val = featureDF.iloc[val_index.tolist()],targetSR[val_index.tolist()]

	#학습진행
	dtc_model.fit(x_train, y_train)

	# 평가 => 분류의 경우 score()메서드 => 정확도 반환
	train_acc=dtc_model.score(x_train,y_train)
	val_acc=dtc_model.score(x_val,y_val)

	accuracys.append([train_acc,val_acc])
	print(f'[{idx}번째] train 정확도 : {train_acc} val 정확도 : {val_acc}')

[1번째] train 정확도 : 1.0 val 정확도 : 0.9666666666666667
[2번째] train 정확도 : 1.0 val 정확도 : 0.9666666666666667
[3번째] train 정확도 : 1.0 val 정확도 : 0.9
[4번째] train 정확도 : 1.0 val 정확도 : 0.9333333333333333
[5번째] train 정확도 : 1.0 val 정확도 : 1.0


In [152]:
# 평균 계산
train_mean = sum([value[0] for value in accuracys])/skFold.n_splits
test_mean = sum([value[1] for value in accuracys])/skFold.n_splits

print(f'Train 정확도 : {train_mean}   val정확도: {test_mean:.2f}')

Train 정확도 : 1.0   val정확도: 0.95


- 교차검증 및 성능평가 동시 진행 함수 
	* => cross_val_score, cross_val_predict
	* => cross_validate

In [153]:
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate

In [154]:
### [1] 전체 DS==> 학습용과 테스트용 DS 분리
x_train,x_test,y_train,y_test=train_test_split(featureDF,
                                              targetSR,
                                              stratify=targetSR)

In [155]:
#### cross_val_predict
predict = cross_val_predict(dtc_model,x_train,y_train,cv=3)

In [156]:
print(f'predict : {predict}')

predict : [1 2 2 2 0 0 1 0 0 0 1 2 0 0 2 2 1 2 0 2 0 0 1 2 0 2 1 0 2 2 0 1 0 0 2 1 0
 1 0 0 0 1 1 1 1 1 2 0 1 0 0 2 2 0 0 1 0 2 2 2 2 2 2 1 2 2 2 0 1 2 0 1 2 1
 2 1 0 1 0 1 1 2 1 2 0 1 2 0 2 2 2 1 0 2 2 0 1 0 1 0 1 1 0 2 1 2 2 1 1 2 1
 0]


In [157]:
#### cross_val_score
cross_val_score(dtc_model,featureDF,targetSR)

array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ])

In [158]:
### cross_validate
result=cross_validate(dtc_model,x_train, y_train,
                      cv=10,
                      return_train_score=True,
                      return_estimator=True)

In [159]:
result

{'fit_time': array([0.00241375, 0.00199318, 0.00099611, 0.00099659, 0.00199318,
        0.00099683, 0.00226784, 0.00103045, 0.00100517, 0.00106716]),
 'score_time': array([0.00126576, 0.00199318, 0.00099659, 0.00100827, 0.00099683,
        0.00099635, 0.00101304, 0.0010078 , 0.00101399, 0.00101876]),
 'estimator': [DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier()],
 'test_score': array([1.        , 1.        , 1.        , 0.90909091, 1.        ,
        1.        , 0.90909091, 0.90909091, 0.90909091, 0.90909091]),
 'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [160]:
resultDF=pd.DataFrame(result).loc[:,['test_score','train_score']]
resultDF

,test_score,train_score
0,1.000000,1.0
1,1.000000,1.0
2,1.000000,1.0
3,0.909091,1.0
4,1.000000,1.0
5,1.000000,1.0
6,0.909091,1.0
7,0.909091,1.0
8,0.909091,1.0
9,0.909091,1.0


- 최적화된 모델 추출

In [161]:
best_model=result['estimator'][3]

- 테스트 데이터로 확인

In [162]:
best_model.predict(x_test)

array([2, 0, 2, 1, 0, 0, 1, 2, 2, 2, 1, 0, 0, 1, 0, 2, 0, 2, 2, 2, 0, 1,
       0, 0, 2, 1, 0, 1, 1, 0, 2, 1, 2, 1, 2, 1, 0, 2])

In [163]:
best_model.score(x_test,y_test)

0.8947368421052632